In [1]:
import numpy as np
import pandas as pd

In [9]:
x = np.random.randint(5, size=[10,4])
y = np.random.randint(5, size=[10,4])

In [26]:
def initMeans(data, n, algo=1):
    if algo==1: # choose n random points
        idx = np.random.choice(range(data.shape[0]), n, False) # no replace
        out = data[idx,]
    if algo==2: # random means points (0,1)
        out = np.random.random([n,data.shape[1]])
        
    return out

################################################################################
def arrayDist(x,y):
    return np.sum((x-y)**2,axis=1)**0.5

In [57]:
def updateMeans(data, means):
    k = means.shape[0] # number of groups
    
    ## Assign each pt to the mean for which it has the shortest distance
    tmpDist = np.zeros([k, data.shape[0]]) # temp array for storing distances
    for n,x in enumerate(means): # loop over all data points
        tmpDist[n] = arrayDist(x,data) # distance from this mean to all pts
    print(tmpDist)
    minDist = tmpDist.argmin(axis=0) # group where data pt has min distance

    ## Calculate new means to be centroid of all the points in the group
    newMeans = np.zeros([k,data.shape[1]]) # new mean points
    for n,x in enumerate(means): # loop over all clusters
        tmp = np.vstack((data[minDist==n,],x)) # concat data pt and centroid
        newMeans[n] = tmp.mean(axis=0) # new mean = centroid of all pts 
    
    return newMeans,minDist

def kMeans(data, k, trace=False, initAlgo=1):
    means = initMeans(data, k, initAlgo) # initialize mean points
    for n in range(5):
        if trace:
            print(means)
        means,grpIdx = updateMeans(data, means)
    return means # return final centroids

In [58]:
x = np.random.randint(5, size=[6,4]) +10 
y = np.random.randint(5, size=[6,4])

z = np.vstack([x,y])

kMeans(z, 2, True, 2)

[[ 0.96608571  0.62364099  0.42399635  0.56558157]
 [ 0.55761653  0.26217732  0.72365564  0.65842646]]
[[ 21.94949902  19.79832029  20.89403435  21.98218816  22.4447161
   23.83391611   4.26721389   3.72618271   4.28255502   3.25154043
    2.2804982    2.97256578]
 [ 22.06984427  19.96661342  21.01524614  22.15814153  22.72147357
   23.97780621   4.65642383   3.32330911   4.69032363   3.52337043
    2.72470087   2.77878546]]
[[ 7.17873506  7.14760373  6.94763603  7.05141651]
 [ 0.51920551  0.42072577  2.24121855  1.88614215]]
[[  9.3843156    7.0640714    8.2654777    9.43954574   9.93937965
   11.09051283   9.40574186  11.60259808  10.07823462   9.69306772
   11.27402218  10.81990931]
 [ 20.56552683  18.55092367  19.50542853  20.75835488  21.58711048
   22.49948034   4.45072799   1.50433146   4.75197474   2.95289661
    3.11815388   1.07491157]]
[[ 10.59696215  10.44965768  10.99251943  11.2930595 ]
 [  1.78845793   1.91724654   2.03445979   1.69802031]]
[[  3.08146958   1.42948086   

array([[ 11.16500572,  10.9983955 ,  11.66470122,  11.99793895],
       [  1.99938326,   2.16593949,   2.00010047,   1.66675808]])

In [ ]:
def ForSelect(data, classVec):
    k = len(np.unique(classVec))
    err = np.Infinity # largest error possible
    
    while 

In [11]:
y*x[1,]

array([[12,  0,  8,  9],
       [ 0,  3, 16, 12],
       [12,  1,  8,  6],
       [ 0,  4,  4,  0],
       [12,  4,  0,  0],
       [ 9,  4,  4,  9]])

## Sources

- [A Modified k-means Algorithm to Avoid Empty Clusters](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.3148&rep=rep1&type=pdf)